In [35]:
import pandas as pd
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

In [2]:
##load datasets

# Load the customer_data
customer_data = pd.read_csv('../../data/user_demographics_v1.csv', parse_dates = True)

# Load the app_purchases
app_purchases = pd.read_csv('../../data/purchase_data_v1.csv', parse_dates = True)

# Print the columns of customer data
print(customer_data.columns)

# Print the columns of app_purchases
print(app_purchases.columns)

Index(['uid', 'reg_date', 'device', 'gender', 'country', 'age'], dtype='object')
Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [3]:
print(customer_data.info())

customer_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
uid         10000 non-null float64
reg_date    10000 non-null object
device      10000 non-null object
gender      10000 non-null object
country     10000 non-null object
age         10000 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 468.9+ KB
None


,uid,reg_date,device,gender,country,age
0,54030035.0,2017-06-29T00:00:00Z,and,M,USA,19
1,72574201.0,2018-03-05T00:00:00Z,iOS,F,TUR,22
2,64187558.0,2016-02-07T00:00:00Z,iOS,M,USA,16
3,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41
4,99231338.0,2017-03-26T00:00:00Z,iOS,M,FRA,59
...,...,...,...,...,...,...
9995,43138741.0,2016-11-23T00:00:00Z,and,M,BRA,16
9996,30326568.0,2016-08-21T00:00:00Z,and,M,BRA,15
9997,92384369.0,2015-08-20T00:00:00Z,and,F,DEU,33
9998,98809962.0,2017-04-08T00:00:00Z,iOS,F,FRA,32


In [4]:
print(app_purchases.info())

app_purchases

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9006 entries, 0 to 9005
Data columns (total 4 columns):
date     9006 non-null object
uid      9006 non-null int64
sku      9006 non-null object
price    9006 non-null int64
dtypes: int64(2), object(2)
memory usage: 281.6+ KB
None


,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599
...,...,...,...,...
9001,2017-02-06,16377492,sku_three_499,499
9002,2017-03-01,16377492,sku_seven_1499,499
9003,2017-05-29,92513925,sku_two_299,299
9004,2017-08-23,92513925,sku_four_599,599


### merging data on UID

In [5]:
purchase_data_uid = pd.merge(app_purchases, customer_data, on=['uid'], how='inner')

print(purchase_data_uid.shape)

purchase_data_uid.head()

(9006, 9)


,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26T00:00:00Z,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26T00:00:00Z,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26T00:00:00Z,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05T00:00:00Z,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05T00:00:00Z,and,M,TUR,17


In [19]:
#cleaning reg_date datetime output 
purchase_data_uid.date = pd.to_datetime(purchase_data_uid.date)

#pd.to_datetime() to use lambda date function
purchase_data_uid.reg_date = pd.to_datetime(purchase_data_uid.reg_date)

# date function transforms data to str -> calling pd.to_datetime() anew
purchase_data_uid.reg_date = pd.to_datetime(purchase_data_uid.reg_date.apply(lambda x: date(x.year, x.month, x.day)))

purchase_data_uid.info()

purchase_data_uid.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9006 entries, 0 to 9005
Data columns (total 9 columns):
date        9006 non-null datetime64[ns]
uid         9006 non-null int64
sku         9006 non-null object
price       9006 non-null int64
reg_date    9006 non-null datetime64[ns]
device      9006 non-null object
gender      9006 non-null object
country     9006 non-null object
age         9006 non-null int64
dtypes: datetime64[ns](2), int64(3), object(4)
memory usage: 703.6+ KB


,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05,and,M,TUR,17


### Calculating KPIs with cohort analysis

In [30]:
# measuring overall average price over 28 day period

In [12]:
purchase_data_uid['date'].max()

Timestamp('2018-04-21 00:00:00')

In [23]:
current_date = datetime(2018, 3, 17)

current_date

datetime.datetime(2018, 3, 17, 0, 0)

In [24]:
max_purchase_date = current_date - timedelta(days=28)

In [25]:
#filtering to include only users who registered before 28 day lapse period
purchase_data_28d = purchase_data_uid[purchase_data_uid.reg_date < max_purchase_date]

,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05,and,M,TUR,17
...,...,...,...,...,...,...,...,...,...
9001,2017-09-16,63245432,sku_five_899,899,2016-12-04,and,F,FRA,20
9002,2017-04-21,36350096,sku_seven_1499,499,2017-04-07,and,M,USA,23
9003,2017-06-04,36350096,sku_three_499,499,2017-04-07,and,M,USA,23
9004,2017-07-12,36350096,sku_one_199,199,2017-04-07,and,M,USA,23


In [27]:
#filtering to include only purchases made before 28 day lapse period
purchase_data_28d = purchase_data_28d[(purchase_data_28d.date <= purchase_data_28d.reg_date + timedelta(days=28))]

purchase_data_28d

,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
19,2016-05-12,22870987,sku_four_599,599,2016-04-20,iOS,F,BRA,26
34,2017-01-12,88736154,sku_five_899,899,2017-01-08,and,F,BRA,19
92,2016-12-08,45588501,sku_four_599,599,2016-11-27,and,F,USA,49
...,...,...,...,...,...,...,...,...,...
8963,2017-12-19,72267850,sku_three_499,499,2017-11-28,iOS,F,TUR,22
8973,2017-12-20,80500054,sku_four_599,599,2017-12-05,and,M,USA,19
8976,2017-05-16,64796589,sku_two_299,299,2017-04-29,and,F,FRA,17
8998,2016-12-29,63245432,sku_four_599,599,2016-12-04,and,F,FRA,20


In [28]:
purchase_data_28d['price'].mean()

414.4237288135593

In [31]:
# measuring average price by cohort over 28 day period

In [32]:
# creating new column "Month_1" when purchasing during 1st month
# using np.where to boolean code

In [38]:
purchase_data_uid['month_1'] = np.where((purchase_data_uid.reg_date < max_purchase_date) &
                                        (purchase_data_uid.date < purchase_data_uid.reg_date + timedelta(days=28)),
                                        purchase_data_uid.price,
                                        np.NaN)

In [40]:
purchase_data_1st_month = purchase_data_uid.groupby(by=['gender', 'device']).agg({'month_1':['mean', 'median'],
                                                                                 'price':['mean', 'median']})

purchase_data_1st_month

month_1              price       
                     mean median        mean median
gender device                                      
F      and     388.204545  299.0  400.747504    299
       iOS     432.587786  499.0  404.435330    299
M      and     413.705882  399.0  416.237308    499
       iOS     433.313725  499.0  405.272401    299

### cleaning data on customer_data
- datetime with standard %Y %m %d
- renaming column reg_date to date in order to merge

In [7]:
#cleaning datetime with standard %Y %m %d
customer_data.reg_date = pd.to_datetime(customer_data.reg_date)

customer_data.reg_date = customer_data.reg_date.apply(lambda x: date(x.year, x.month, x.day))

customer_data.rename(columns={'reg_date':'date'}, inplace=True)

customer_data.date = pd.to_datetime(customer_data.date)

customer_data

,uid,date,device,gender,country,age
0,54030035.0,2017-06-29,and,M,USA,19
1,72574201.0,2018-03-05,iOS,F,TUR,22
2,64187558.0,2016-02-07,iOS,M,USA,16
3,92513925.0,2017-05-25,and,M,BRA,41
4,99231338.0,2017-03-26,iOS,M,FRA,59
...,...,...,...,...,...,...
9995,43138741.0,2016-11-23,and,M,BRA,16
9996,30326568.0,2016-08-21,and,M,BRA,15
9997,92384369.0,2015-08-20,and,F,DEU,33
9998,98809962.0,2017-04-08,iOS,F,FRA,32


In [8]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
uid        10000 non-null float64
date       10000 non-null datetime64[ns]
device     10000 non-null object
gender     10000 non-null object
country    10000 non-null object
age        10000 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 468.9+ KB


In [9]:
app_purchases.date = pd.to_datetime(app_purchases.date)

print(app_purchases.info())

app_purchases.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9006 entries, 0 to 9005
Data columns (total 4 columns):
date     9006 non-null datetime64[ns]
uid      9006 non-null int64
sku      9006 non-null object
price    9006 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 281.6+ KB
None


,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


### Creating new df date_purchase_data : double merge of customer & app purchases on both UID & Date

In [10]:
# Merge on the 'uid' and 'date' field
date_purchase_data = app_purchases.merge(customer_data, on=['uid', 'date'], how='inner')

# Examine the results 
print(date_purchase_data.info())


date_purchase_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 8 columns):
date       35 non-null datetime64[ns]
uid        35 non-null int64
sku        35 non-null object
price      35 non-null int64
device     35 non-null object
gender     35 non-null object
country    35 non-null object
age        35 non-null int64
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 2.5+ KB
None


,date,uid,sku,price,device,gender,country,age
0,2016-03-30,94055095,sku_four_599,599,iOS,F,BRA,16
1,2015-10-28,69627745,sku_one_199,199,and,F,BRA,18
2,2017-02-02,11604973,sku_seven_1499,499,and,F,USA,16
3,2016-06-05,22495315,sku_four_599,599,and,F,USA,19
4,2018-02-17,51365662,sku_two_299,299,iOS,M,TUR,16
